# 📓 Notebook Overview: Daily Briefing Summarizer Agent
This notebook builds an AI agent using LangChain that generates a daily briefing by combining information from Google Calendar and Slack.

# 🗂️ Notebook Structure
## 📦 1. Imports and Setup
- Loads necessary libraries (LangChain, LLMs, calendar/slack functions).
- Make sure your .env, credentials.json files are properly configured.

## 📅 2. Get Google Calendar Events
- Fetches all events for the current day using the Google Calendar API.
- Data is shown in plaintext format for inspection.

## 💬 3. Get Slack Messages
- Retrieves the most recent Slack messages from a specific channel (default: last 60 minutes).
- Slack bot must have permission and be invited to the channel.

## 🧠 4. Summarize Both Sources with LLM
- Combines calendar and Slack inputs into a single structured summary.
- Uses a GPT-based LLM via LangChain and a custom prompt template.
- The output includes a markdown-style summary with headings.

In [ ]:
from __future__ import print_function
import datetime
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import os
from datetime import timedelta

import getpass

from langchain.tools import BaseTool


# Google Calendar

In [19]:
# If modifying these scopes, delete the token.json file
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

def get_calendar_events():
    creds = None
    # token.json stores the user's access/refresh tokens after first login
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If no (valid) creds, let user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('calendar', 'v3', credentials=creds)

    # Call API to get upcoming events
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' = UTC time
    end = (datetime.datetime.utcnow() + datetime.timedelta(days=1)).isoformat() + 'Z'

    events_result = service.events().list(
        calendarId='primary', timeMin=now, timeMax=end,
        maxResults=10, singleEvents=True,
        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        return "No upcoming events found."
    
    output = []
    for event in events:
        title = event.get('summary', 'No Title')
        if title == 'Home':
            continue
        start = event['start'].get('dateTime', event['start'].get('date'))
        end = event['end'].get('dateTime', event['end'].get('date'))
        output.append(f"{start} - {end} — {title}")

    return " ".join(output)


In [50]:
class CalendarTool(BaseTool):
    name: str = "calendar_tool"
    description: str = "Get today's calendar events from Google Calendar."

    def _run(self, query: str):
        return get_calendar_events()

    def _arun(self, query: str):
        raise NotImplementedError("Async not supported")


# Slack

In [80]:
# Load your bot token
SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
client = WebClient(token=SLACK_BOT_TOKEN)

def get_recent_messages(channel_id: str, minutes_back: int = 600):
    try:
        # Timestamp for messages from X minutes ago until now
        now = datetime.datetime.utcnow()
        oldest = (now - timedelta(minutes=minutes_back)).timestamp()

        response = client.conversations_history(
            channel=channel_id,
            oldest=str(oldest)
        )

        messages = response["messages"]
        print(messages)
        return [
            f"[{datetime.datetime.utcfromtimestamp(float(m['ts']))}] {m.get('user', 'bot')} - {m.get('text', '')}"
            for m in messages
        ]
    except SlackApiError as e:
        return f"Error: {e.response['error']}"


In [81]:
class SlackTool(BaseTool):
    name: str = "slack_tool"
    description: str = "Fetch recent Slack messages"

    def _run(self, query: str):
        return "\n".join(get_recent_messages(channel_id="C0912N8JV8A", minutes_back=600))

    def _arun(self, query: str):
        raise NotImplementedError("Async not supported")


# Model

In [40]:
tools = [CalendarTool, SlackTool]

In [41]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4", model_provider="openai")

Enter API key for OpenAI:  ········


In [82]:
from langchain.agents import Tool, initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)

tools = [
    Tool(
        name="Calendar Tool",
        func=CalendarTool().run,
        description="Use this to get today's calendar events."
    ),
    Tool(
        name="Slack Tool",
        func=SlackTool().run,
        description="Use this to get recent Slack messages."
    ),
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=False
)

response = agent.run("What are my todos based on google calendar and my slack messages and when I should do them (also the name of the day)? Order them pls and format datetime as YYYY-MM-DD HH:MM.")
print(response)


C:\Users\Vacuumlabs\AppData\Local\Temp\ipykernel_15088\3458627944.py:30: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' = UTC time
C:\Users\Vacuumlabs\AppData\Local\Temp\ipykernel_15088\3458627944.py:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = (datetime.datetime.utcnow() + datetime.timedelta(days=1)).isoformat() + 'Z'
C:\Users\Vacuumlabs\AppData\Local\Temp\ipykernel_15088\3884348102.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.d

[{'user': 'U019UJU02UD', 'type': 'message', 'ts': '1750150508.067049', 'client_msg_id': '78f23022-d931-42b9-a55a-c4c277e1011b', 'text': 'Hi summarizer, please finish AI academy by EOD', 'team': 'T026LE24D', 'blocks': [{'type': 'rich_text', 'block_id': '6lD+B', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'Hi summarizer, please finish AI academy by EOD'}]}]}]}, {'subtype': 'channel_join', 'user': 'U091TFPRUBW', 'text': '<@U091TFPRUBW> has joined the channel', 'inviter': 'U019UJU02UD', 'type': 'message', 'ts': '1750150458.159129'}]


C:\Users\Vacuumlabs\AppData\Local\Temp\ipykernel_15088\3884348102.py:20: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  f"[{datetime.datetime.utcfromtimestamp(float(m['ts']))}] {m.get('user', 'bot')} - {m.get('text', '')}"


1. 2025-06-17 14:00:00 - Kids
2. 2025-06-17 23:59:00 - Finish AI academy (EOD)
3. 2025-06-18 09:30:00 - Karol / Jucus - Vesna onboarding
4. 2025-06-18 11:00:00 - Karol / Janos sync (AUXM-3)
